In [3]:
#loading takes a while, don't run this if testing
print('loading matlab engine...')
import matlab.engine
eng = matlab.engine.start_matlab()
print('matlab engine loaded')

loading matlab engine...
matlab engine loaded


## Run the cell below to begin analyzing.

Check the output; it will update as images are analyzed.

To stop the analysis, press the interrupt kernel/stop button at the very top. You can restart the analysis by simply running the cell below again. The console output will be cleared, but not to worry, it's all in the log file.

In [2]:
breadboard_repo_path =  r'D:\Fermidata1\enrico\breadboard-python-client\\'
import ipywidgets as widgets
from ipywidgets import interact
from measurement_directory import *
import sys
sys.path.insert(0, breadboard_repo_path)
from breadboard import BreadboardClient
bc = BreadboardClient(config_path='API_CONFIG_fermi1.json') # enter your path to the API_config
import os
import time
import datetime
import shutil
import warnings
import enrico_bot
from image_watchdog import getFileList

# import pandas as pd
# from numpy import isnan
# analysis_type = 'fake analysis'

# define analysis_function which accepts image filepath and returns dictionary of {analyzed_var_name:analysis_value}
if analysis_type == 'fake analysis':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,2))
        return {'fake analysis 1': randint(0,42), 'fake analysis 2': randint(0,42)}
    analyzed_var_names = ['fake analysis 1', 'fake analysis 2']
elif analysis_type == 'fake analysis 2':
    from random import randint
    def analysis_function(filepath):
        time.sleep(randint(0,5))
        return {'fake analysis 3': randint(0,42), 'fake analysis 4': randint(0,42)}
    analyzed_var_names = ['fake analysis 3', 'fake analysis 4']
elif analysis_type == 'ycam':
    analyzed_var_names = ['bareNcntAverageMarqueeBoxValues', 
                          'COMX', 'COMY'] #TODO set MATLAB analyzed_var_names properly
    from matlab_wrapper import getYcamAnalysis
    def analysis_function(filepath, previous_settings = None):
        if previous_settings is None:
            matlab_dict = getYcamAnalysis(eng, filepath)
        else:
#             print('previous settings: ' + previous_settings)
#             return {'fakematlab1': randint(0,42), 'fakematlab2': randint(0,42)}, 'foo1'
            matlab_dict = getYcamAnalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
                                           normBox = previous_settings['normBox'])
        analysis_dict, settings = matlab_dict['analysis'], matlab_dict['settings']
        return analysis_dict, settings
elif analysis_type == 'zcam_dual_imaging':
    analyzed_var_names = ['K_bareNcntAverageMarqueeBoxValues', 'Na_bareNcntAverageMarqueeBoxValues',
                          'Na_COMX', 'Na_COMY'] #TODO set MATLAB analyzed_var_names properly
    from dual_imaging_wrapper import getDualImagingAnalysis
    def analysis_function(filepath, previous_settings = None):
        if previous_settings is None:
            matlab_dict = getDualImagingAnalysis(eng, filepath)
#         else:
#             matlab_dict = getMATLABanalysis(eng, filepath, marqueeBox = previous_settings['marqueeBox'],
#                                            normBox = previous_settings['normBox'])
        analysis_dict = matlab_dict['analysis']
        settings = None
        return analysis_dict, settings 

warnings.filterwarnings(
    "ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings(
    "ignore", "Could not find appropriate MS Visual C Runtime")
bc = BreadboardClient(config_path='API_CONFIG_fermi1.json')

def analyze_image(image_filename, previous_settings = None, output_previous_settings = True):
    #TODO adapt for triple imaging later
    abs_image_path = os.path.join(os.path.join(os.getcwd(), watchfolder), image_filename)
    settings = None
    print('{file} analyzing: '.format(file=image_filename))
    if analysis_type == 'fake analysis' or analysis_type == 'fake analysis 2':
        analysis_dict = analysis_function(image_filename)
    elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging') and previous_settings is None:
        analysis_dict, settings = analysis_function(abs_image_path)#no marquee box yet
    elif (analysis_type == 'ycam' or analysis_type == 'zcam_dual_imaging'):
        analysis_dict, settings = analysis_function(abs_image_path, previous_settings)
    if not output_previous_settings:
        settings = None #forces user to select new marquee box for each shot
    for key in analyzed_var_names:
        print(key, analysis_dict[key])
    return analysis_dict, settings

def main():
    # Global settings
    refresh_time = 1  # seconds
    previous_settings = None
    print("\n\n Watching this folder for changes: " + watchfolder + "\n\n")
    unanalyzed_files = []
    done_files = []
    append_mode = True

    # Main Loop
    while True:
        if not os.path.exists(watchfolder):
            time.sleep(refresh_time)
            continue
        else:
            files, _ = getFileList(watchfolder)
            fresh_files = sorted(list(set(files).difference(set(done_files)).difference(set(unanalyzed_files))))
            unanalyzed_files += fresh_files #push newest files to top of stack
        
        for file in reversed(unanalyzed_files): #start from top of stack
            run_id = run_id_from_filename(file)
            if append_mode:
                run_dict = bc._send_message('get', '/runs/' + str(run_id) + '/').json()
                if set(analyzed_var_names).issubset(set(run_dict.keys())):
                    popped_file = [unanalyzed_files.pop()]
                    done_files += popped_file
                    continue
            analysis_dict, previous_settings = analyze_image(file, previous_settings)
            resp = bc.append_analysis_to_run(run_id, analysis_dict)
            popped_file = [unanalyzed_files.pop()]
            done_files += popped_file
            print('\n')

        time.sleep(refresh_time)
# try:
#     main()
# except:
#     enrico_bot.post_message('analysis crashed, restart it.')
#     pass


def set_analysis_type(analysis_mode):
    global analysis_type
    analysis_type = analysis_mode
    print('analysis mode is {a_type} \n'.format(a_type = analysis_type))

mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2', 'ycam', 'zcam_dual_imaging'])
measurement_menu = widgets.Dropdown(options = todays_measurements(),
                                   description = 'today\'s measurements' )
load_measurement_button = widgets.Button(description = 'select measurement')
def load_measurement(button):
    global watchfolder
    watchfolder = measurement_directory(measurement_name = measurement_menu.value)
    print(watchfolder)
    main()
load_measurement_button.on_click(load_measurement)
widgets.HBox([measurement_menu, load_measurement_button])

interactive(children=(Dropdown(description='analysis_mode', options=('fake analysis', 'fake analysis 2', 'ycam…

Dropdown(description="today's measurements", options=('debugging',), value='debugging')

Button(description='select measurement', style=ButtonStyle())

08\200811\debugging


 Watching this folder for changes: 08\200811\debugging


222050_0.spe analyzing: 
fake analysis 1 37
fake analysis 2 40
run_id 222050 analyzed: {'fake analysis 1': 37, 'fake analysis 2': 40}

FBinGauss: 
80.3
SeqMode: 
0.0
repetition: 
1.0
PAtime: 
10.0
AOMPA: 
285.0
PAlogic: 
0.0


222049_0.spe analyzing: 
fake analysis 1 39
fake analysis 2 31
run_id 222049 analyzed: {'fake analysis 1': 39, 'fake analysis 2': 31}

FBinGauss: 
80.3
SeqMode: 
1.0
repetition: 
1.0
PAtime: 
10.0
AOMPA: 
285.0
PAlogic: 
1.0


222047_0.spe analyzing: 
fake analysis 1 3
fake analysis 2 12
run_id 222047 analyzed: {'fake analysis 1': 3, 'fake analysis 2': 12}

FBinGauss: 
80.3
SeqMode: 
0.0
repetition: 
1.0
PAtime: 
10.0
AOMPA: 
275.0
PAlogic: 
0.0


222045_0.spe analyzing: 
fake analysis 1 23
fake analysis 2 24
run_id 222045 analyzed: {'fake analysis 1': 23, 'fake analysis 2': 24}

FBinGauss: 
80.3
SeqMode: 
1.0
repetition: 
1.0
PAtime: 
10.0
AOMPA: 
275.0
PAlogic: 
1.0


222043_0.spe an

In [ ]:
#testing cleaned up analysis_logger.py file below

from analysis_logger import main
mode_select_widget = interact(set_analysis_type,analysis_mode=['fake analysis' , 'fake analysis 2', 'ycam', 'zcam_dual_imaging'])
measurement_menu = widgets.Dropdown(options = todays_measurements(),
                                   description = 'today\'s measurements' )
load_measurement_button = widgets.Button(description = 'select measurement')
def load_measurement(button):
    global watchfolder
    watchfolder = measurement_directory(measurement_name = measurement_menu.value)
    print(watchfolder)
    main(analysis_type, watchfolder)
load_measurement_button.on_click(load_measurement)
widgets.HBox([measurement_menu, load_measurement_button])